<a href="https://colab.research.google.com/github/matthew110395/12004210_DataAnalytics/blob/main/12004210_DAOTW_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
This assignment builds on the New York taxi problem identified in assignment 1, where the City of New York is looking for a way to accurately predict the number of collisions on a particular day of the week. Within this document two different types of machine learning models will be utilised to predict the number of collisions. The linear relationships identified in assignment 1 will be used to create a linear regression model. A Deep Learning Neural Network (DNN) will be used to predict the number of collisions where the relationship is not linear. 

#Imports

In [1]:
import pandas as pd
import numpy as np

!pip install tensorflow==1.15.2
import tensorflow as tf
# needed for high-level file management
import shutil  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110.5 MB 37 kB/s 
     |████████████████████████████████| 3.8 MB 31.7 MB/s 
     |████████████████████████████████| 503 kB 44.5 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=ddd8b77ed2024a566910e240d182a80e9fbbdaad96847689fe847880dc43387d
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting un

#Linear Regressor


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/matthew110395/12004210_DataAnalytics/main/prcp_clean.csv', index_col=0, )
print(df[:6])

   day  year  mo  da collision_date  NUM_COLLISIONS  temp  dewp     slp  \
1    5  2020   1  24     2020-01-24             524  37.3  33.7  1028.5   
2    2  2021   1  12     2021-01-12             278  37.0  29.1  1019.0   
3    5  2021   1  22     2021-01-22             254  36.5  28.4  1003.1   
4    3  2021   1  27     2021-01-27             262  34.6  33.8  1012.8   
5    2  2021   1  26     2021-01-26             263  31.9  23.4  1016.9   
6    1  2022   1  24     2022-01-24             237  34.5  23.8  1010.6   

   visib  ...   max   min  prcp   sndp  fog  rain_drizzle  snow_ice_pellets  \
1    6.5  ...  46.0  19.9  0.00  999.9    1             0                 0   
2   10.0  ...  44.1  21.0  0.00  999.9    0             0                 0   
3   10.0  ...  44.1  19.9  0.00  999.9    0             0                 0   
4    8.0  ...  41.0  28.9  0.25  999.9    1             1                 0   
5    9.0  ...  37.9  21.0  0.00  999.9    1             0                 0   


In [3]:
df_prcp = df.drop(columns=['collision_date', 'temp', 'dewp', 'slp','visib','max','min','sndp','wdsp','mxpsd','gust','thunder','tornado_funnel_cloud'])
df_prcp = df_prcp.loc[df_prcp["year"] != 2012]
df_prcp = df_prcp.loc[df_prcp["year"] < 2020]
cols = df_prcp['NUM_COLLISIONS']
df_prcp = df_prcp.drop(columns=['NUM_COLLISIONS'])
df_prcp.insert(loc=9, column='NUM_COLLISIONS', value=cols)
print(df_prcp[:6])
df_prcp.describe()

    day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  \
49    4  2016   1  28  0.09    0             0                 0     0   
51    5  2014   1  17  0.00    1             0                 0     0   
54    1  2016   1  25  0.02    0             0                 0     0   
55    5  2016   1  29  0.00    0             0                 0     0   
58    5  2017   1  20  0.00    0             0                 0     0   
59    7  2013   1  13  0.01    1             0                 0     0   

    NUM_COLLISIONS  
49             681  
51             589  
54             658  
55             645  
58             605  
59             373  


,day,year,mo,da,prcp,fog,rain_drizzle,snow_ice_pellets,hail,NUM_COLLISIONS
count,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000,2539.000000
mean,3.998425,2015.989366,6.518708,15.745569,0.122588,0.253249,0.375345,0.085467,0.000394,599.135093
std,2.003542,1.996126,3.455211,8.803199,0.329143,0.434958,0.484307,0.279630,0.019846,100.299164
min,1.000000,2013.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,188.000000
25%,2.000000,2014.000000,4.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,531.000000
50%,4.000000,2016.000000,7.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000
75%,6.000000,2018.000000,10.000000,23.000000,0.060000,1.000000,1.000000,0.000000,0.000000,665.000000
max,7.000000,2019.000000,12.000000,31.000000,3.760000,1.000000,1.000000,1.000000,1.000000,1161.000000


In [4]:
# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df_prcp.iloc[np.random.permutation(len(df_prcp))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:-1]


# print out the first 6 rows of predictors.
print(predictors[:6])
print(shuffle[:6])

      day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail
2127    5  2017   7   7  0.00    1             1                 0     0
495     1  2015   2   9  0.04    0             1                 0     0
295     4  2019   1  24  0.00    0             1                 0     0
546     6  2019   2   2  0.00    0             0                 0     0
2904    7  2013  10  13  0.00    0             0                 0     0
545     7  2015   2   1  0.00    0             0                 0     0
      day  year  mo  da  prcp  fog  rain_drizzle  snow_ice_pellets  hail  \
2127    5  2017   7   7  0.00    1             1                 0     0   
495     1  2015   2   9  0.04    0             1                 0     0   
295     4  2019   1  24  0.00    0             1                 0     0   
546     6  2019   2   2  0.00    0             0                 0     0   
2904    7  2013  10  13  0.00    0             0                 0     0   
545     7  2015   2   1  0.00    

In [5]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

2127    674
495     547
295     592
546     577
2904    502
545     361
Name: NUM_COLLISIONS, dtype: int64


In [6]:
SCALE_COLLISIONS=1161

In [47]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
print(trainsize)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize
print(testsize)
# Define the number of input values (predictors)
nppredictors = 9
# Define the number of output values (targets)
noutputs = 1

2031
508


In [55]:
# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', optimizer=tf.train.AdamOptimizer(learning_rate=0.001), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))
print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_COLLISIONS, steps=10000)
preds = estimator.predict(x=predictors[trainsize:].values)
#print(preds)
predslistscale = preds['scores']*SCALE_COLLISIONS

pred = format(str(predslistscale))
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])
rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f495da5fd10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Create CheckpointSaverHook.


starting to train


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/linear_regression_trained_model/model.ckpt.
INFO:tensorflow:loss = 0.28681284, step = 1
INFO:tensorflow:global_step/sec: 789.909
INFO:tensorflow:loss = 0.005889251, step = 101 (0.133 sec)
INFO:tensorflow:global_step/sec: 859.739
INFO:tensorflow:loss = 0.00927359, step = 201 (0.114 sec)
INFO:tensorflow:global_step/sec: 918.532
INFO:tensorflow:loss = 0.0074369605, step = 301 (0.110 sec)
INFO:tensorflow:global_step/sec: 821.327
INFO:tensorflow:loss = 0.00594569, step = 401 (0.121 sec)
INFO:tensorflow:global_step/sec: 919.42
INFO:tensorflow:loss = 0.006601665, step = 501 (0.110 sec)
INFO:tensorflow:global_step/sec: 867.937
INFO:tensorflow:loss = 0.0063313283, step = 601 (0.115 sec)
INFO:tensorflow:global_step/sec: 862.334
INFO:tensorflow:loss = 0.0074583776, step = 701 (0.113 sec)
INFO:tensorflow:global_step/sec: 894.436


{'scores': array([0.5141025 , 0.56370085, 0.5293338 , 0.50650656, 0.5460061 ,
       0.46276766, 0.48901325, 0.5045964 , 0.55767345, 0.5474755 ,
       0.47696412, 0.5719989 , 0.499372  , 0.54435164, 0.496742  ,
       0.52334034, 0.5130221 , 0.5576516 , 0.5660157 , 0.50187653,
       0.49803287, 0.5275236 , 0.5175727 , 0.5518675 , 0.46650067,
       0.55129665, 0.526218  , 0.47557315, 0.5527711 , 0.52379066,
       0.5528509 , 0.5527703 , 0.501649  , 0.5582163 , 0.47037572,
       0.55934405, 0.5168263 , 0.5386811 , 0.46069485, 0.5659551 ,
       0.49748445, 0.51326185, 0.55960935, 0.5027483 , 0.5531555 ,
       0.5192229 , 0.5500306 , 0.5822944 , 0.5859781 , 0.5465521 ,
       0.5313575 , 0.5444995 , 0.58136153, 0.5042304 , 0.46689093,
       0.55589783, 0.5206485 , 0.5478087 , 0.5983786 , 0.59213996,
       0.56438524, 0.558204  , 0.49582595, 0.5353597 , 0.5774194 ,
       0.5115516 , 0.5272639 , 0.54816175, 0.55420434, 0.54816544,
       0.5269321 , 0.47668824, 0.51236   , 0.496718

In [96]:
print(len(predictors[trainsize:].values))
print(len(targets[trainsize:].values.reshape(testsize, noutputs)/SCALE_COLLISIONS))

#testd = pd.DataFrame.from_records(predictors[trainsize:].values,columns=['day','year','month','da','prcp','fog','rain','snow','hail'])
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors[trainsize:].values)))
preds = estimator.predict(x=predictors[trainsize:].values)

print(preds['scores'])
print(targets[trainsize:].values/SCALE_COLLISIONS)

testdf = pd.DataFrame.from_dict(data={
    'pred': preds['scores'],
    'actual': targets[trainsize:].values/SCALE_COLLISIONS
})

testdf['diff'] = testdf['actual'] - testdf['pred']

error=testdf['diff'].mean()*SCALE_COLLISIONS
avgcol=targets[trainsize:].mean()
print('The trained model has an aproximate error rate of {0} which equates to {1}%'.format(error, round((error/avgcol)*100),1));


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f495d737990>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model/model.ck

508
508
[0.5141025  0.56370085 0.5293338  0.50650656 0.5460061  0.46276766
 0.48901325 0.5045964  0.55767345 0.5474755  0.47696412 0.5719989
 0.499372   0.54435164 0.496742   0.52334034 0.5130221  0.5576516
 0.5660157  0.50187653 0.49803287 0.5275236  0.5175727  0.5518675
 0.46650067 0.55129665 0.526218   0.47557315 0.5527711  0.52379066
 0.5528509  0.5527703  0.501649   0.5582163  0.47037572 0.55934405
 0.5168263  0.5386811  0.46069485 0.5659551  0.49748445 0.51326185
 0.55960935 0.5027483  0.5531555  0.5192229  0.5500306  0.5822944
 0.5859781  0.5465521  0.5313575  0.5444995  0.58136153 0.5042304
 0.46689093 0.55589783 0.5206485  0.5478087  0.5983786  0.59213996
 0.56438524 0.558204   0.49582595 0.5353597  0.5774194  0.5115516
 0.5272639  0.54816175 0.55420434 0.54816544 0.5269321  0.47668824
 0.51236    0.49671885 0.52924085 0.58760697 0.52983314 0.4971317
 0.5695469  0.5349508  0.5067138  0.57710415 0.5700293  0.5200887
 0.51930255 0.5406472  0.51061755 0.5496228  0.5225133  0.4975